#### Start a simple Spark Session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('df-project').getOrCreate()

#### Load the CSV File, have Spark infer the data types.

In [ ]:
df = spark.read.format('csv').option('header', 'true').load('../data/original.csv')
df.show()
df.printSchema()

In [ ]:
df = spark.read.csv('../data/original.csv', header=True, inferSchema=True)

In [ ]:
#### If city is null, then set city to 'Unknown'
from pyspark.sql.functions import *
c_df = df.withColumn('city', when(df.City.isNull(), 'Unknown').otherwise(df.City))
c_df.show()

In [ ]:
# If the Job title is null, then delete the row
# df = df.na.drop(subset=['Job Title'])
df_title = df.filter(df.JobTitle.isNotNull())

### If Salary is null then replace it with Average salary

In [ ]:
df_sal = df.withColumn('Salary_fix', df.Salary.substr(2,100).cast('float'))
df_sal.show()

#### What are the column names?

In [ ]:
df.columns

#### What does the Schema look like?

In [ ]:
df.printSchema()

#### Print out the first 5 columns.

In [ ]:
df.head(5)

#### Use describe() to learn about the DataFrame.

In [ ]:
df.describe().show()

#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [ ]:
desc = df.describe()
desc.show()
desc.printSchema()

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
desc.select(desc.summary,
                  format_number(desc.Open.cast('float'),2).alias('Open'),
                  format_number(desc.High.cast('float'),2).alias('High'),
                  format_number(desc.Low.cast('float'),2).alias('Low'),
                  format_number(desc.Close.cast('float'),2).alias('Close'),
                  format_number(desc.Volume.cast('int'), 1).alias('Volume'),
                  format_number(desc['Adj Close'].cast('float'), 2).alias('AdjClose')).show()


#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
hv = df.withColumn('HV Ratio', df['High']/df['Volume'])
hv.select('HV Ratio').show()

#### What day had the Peak High in Price?

In [ ]:
df.orderBy(df["High"].desc()).head(1)[0][0]

#### What is the mean of the Close column?

In [ ]:
df.select(['Close']).agg({'Close':'avg'}).collect()[0]

#### What is the max and min of the Volume column?

In [ ]:
from pyspark.sql.functions import max, min

In [ ]:
df.select(max("Volume"), min("Volume")).show()

#### How many days was the Close lower than 60 dollars?

In [ ]:
df.filter('Close < 60').count()

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
df.filter('High>80').count()/df.select('High').count() * 100

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [ ]:
from pyspark.sql.functions import corr
df.select(corr('Volume', 'High')).show()

#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import year
df.withColumn("Year", year(df["Date"])).groupBy('Year').max().select('Year', "max(High)").show()

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
from pyspark.sql.functions import month
df = df.withColumn("Month", month('Date'))
(df.select('Month', 'Close').groupBy('Month').avg().
                    select('Month','avg(Close)')).orderBy('Month').show()